<h3> BT4222 Project - Toxic Comments Classifier (Chain Classifier) </h3>

In this notebook, we will train the clain classifier which investigates 720 different permutations using logistic regresison as our baseline model. For more information please refer to the report.

In [ ]:
import pandas as pd
import numpy as np
import os
import nltk
import re


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.tokenize import RegexpTokenizer,sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier,ExtraTreesClassifier
from sklearn import tree, manifold,datasets
from sklearn.model_selection import KFold, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import tree
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc, make_scorer , recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV,KFold, cross_val_score
from sklearn.cluster import KMeans
#import lightgbm as lgb
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.model_selection import validation_curve
from sklearn.feature_selection import SelectKBest, chi2, f_classif

from sklearn.decomposition import PCA

from hyperopt import hp
from hyperopt.pyll import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing

from sklearn.multioutput import ClassifierChain
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [7]:
df_train = pd.read_csv("cleaned_train.csv")

In [8]:
# filter out -1
df_test_label = pd.read_csv("test_labels.csv")
df_test_label = df_test_label[df_test_label.toxic != -1]
test_id = list(df_test_label.id)
df_test_label.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


In [9]:
df_test = pd.read_csv("cleaned_test.csv")
df_test = df_test[df_test.id.isin(test_id)]
df_test.head()

,id,comment_text
5,0001ea8717f6de06,thank understanding think highly would revert ...
7,000247e83dcc1211,dear god site horrible
11,0002f87b16116a7f,somebody invariably try add religion really me...
13,0003e1cccfd5a40a,says right type type institution needed case t...
14,00059ace3e3e9a53,adding new product list make sure relevant add...


In [10]:
df_train['comment_text'].fillna("unknown", inplace=True)
df_test['comment_text'].fillna("unknown", inplace=True)

In [11]:
label = list(df_train.columns[2:])

In [12]:
train_y = df_train[label]
test_y = df_test_label[label]

In [13]:
tfidf = TfidfVectorizer(strip_accents='unicode',
                              analyzer='word',
                              norm='l2',
                              ngram_range=(1,1),
                              stop_words="english",
                              lowercase=True,
                             )

df = pd.concat([df_train, df_test], axis=0)
data = tfidf.fit(df['comment_text'])
tf_train_x = tfidf.transform(df_train['comment_text'])
tf_X_test = tfidf.transform(df_test['comment_text'])

In [14]:
from itertools import permutations 
orders = list(permutations(range(0, 6))) 
len(orders)

720

In [35]:
from tqdm import tqdm_notebook as tqdm
base_lr = LogisticRegression(random_state=1234)
chains =[ClassifierChain(base_lr, order=list(i), random_state=1234) for i in orders]

best = 0
bestModel = None

for j,chain in enumerate(tqdm(chains)):
    chain.fit(tf_train_x,train_y) 
    chain_pred = chain.predict_proba(tf_X_test)
    curr_score = roc_auc_score(np.array(test_y), chain_pred, average='weighted')
    if curr_score > best:
        print(curr_score)
        best = curr_score
        bestModel = chain


0.9591597184931395
0.9591894464860267
0.9592066103526158
0.9593398677366043
0.9593694228057056
0.9593863299499032
0.9595222002733474
0.9595456813700036
0.9595628158686778



In [36]:
chain

ClassifierChain(base_estimator=LogisticRegression(C=1.0, class_weight=None,
                                                  dual=False,
                                                  fit_intercept=True,
                                                  intercept_scaling=1,
                                                  l1_ratio=None, max_iter=100,
                                                  multi_class='warn',
                                                  n_jobs=None, penalty='l2',
                                                  random_state=1234,
                                                  solver='warn', tol=0.0001,
                                                  verbose=0, warm_start=False),
                cv=None, order=[5, 4, 3, 2, 1, 0], random_state=1234)